## 과제 2: 신경망을 사용하여 다층 퍼셉트론 모델 생성 및 실행

이번 과제에서는 [Apache MXNet](https://mxnet.apache.org/) 과 같은 강력한 ML 라이브러리에서 나온 추상화된 방법을 사용하지 않고 신경망을 생성합니다. 하지만, MXNet의 몇몇 기본 함수를 사용하면 이러한 신경망을 쉽게 모델링할 수 있습니다.

가장 단순한 신경망으로 쉽게 표현할 수 있는 **로지스틱 회귀 분석** 문제에 초점을 맞춥니다. 결과\(레이블\)가 알려진 모조 데이터 세트를 생성합니다. 필기 숫자의 28x28 흑백 사진을 반으로 자른 사진으로 구성된 [MNIST 데이터 세트](http://yann.lecun.com/exdb/mnist/)를 사용합니다.

![](mnistdigits.gif)

숫자를 감지하기 위해 4계층 신경망을 생성합니다.


**Shift+Enter** 키를 눌러 셀을 실행합니다. 셀 실행이 완료되면 셀 좌측의 숫자가 **In [*]:** 에서 **In [1]** 로 변경됩니다.

In [ ]:
# Import dependencies
from __future__ import print_function
import mxnet as mx
import numpy as np
from mxnet import nd, autograd
print("Dependencies imported")

MXNet에서 GPU를 사용하여 지정하려면, 다음번 셀에서 다음 명령을 실행합니다. CPU를 대신 사용하려면 아래 셀을 실행하기 전에 CPU 줄의 주석 처리를 해제하고 GPU 줄을 주석 처리합니다.

In [ ]:
# Use a GPU with MXNet
ctx = mx.gpu()

# Use a CPU with MXNet
# ctx = mx.cpu()

다층 퍼셉트론 신경망을 위해 [MNIST 이미지 데이터 세트](http://yann.lecun.com/exdb/mnist/)를 사용합니다. 데이터 세트를 다운로드하려면, 다음번 셀에서 다음을 입력합니다.

In [ ]:
# Get the MNIST image dataset
mnist = mx.test_utils.get_mnist()

신경망에 대한 파라미터를 정의하려면 다음번 셀에 다음 코드를 붙여넣습니다.

In [ ]:
# Number of inputs: 1-dimensional input consisting of a single image (28x28 pixels)
num_inputs = 784

# Number of outputs: Number of outputs to be predicted by the network (digits 0-9)
num_outputs = 10

# Batch size: Number of images processed in a single batch
batch_size = 64

파라미터를 정의한 후에 데이터 세트를 교육 데이터와 테스트 데이터로 나눌 수 있습니다. Gluon API를 사용하여 DataLoader를  미니 배치로 데이터 세트를 반복합니다. DataLoader는 Dataset에서 샘플의 미니 배치를 생성하는 데 사용되며 이러한 배치를 반복하기 위한 편리한 반복기 인터페이스를 제공합니다. 컴퓨팅을 병렬로 수행할 수 있기 때문에 일반적으로 신경망을 통해 데이터의 미니 배치를 전달하는 것이 한 번에 단일 샘플보다 훨씬 효율적입니다.

Gluon은 `mxnet.gluon.data.vision.MNIST`를 사용하여 MNIST 데이터 세트를 다운로드할 수 있는 편리한 API를 제공합니다.

In [ ]:
# Split the dataset into training data and test data

def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)

train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),batch_size, shuffle=False)

다음으로 **숨겨진 뉴런 수** 및 **가중치 배율** 등의 유용한 파라미터를 정의합니다.

In [ ]:
# Number of hidden neurons
num_hidden = 128

# Weights scale
weight_scale = .01

첫 번째 계층에 대한 파라미터(**가중치** 및 **편중치**)를 할당하려면, 다음번 셀에서 다음 코드를 실행합니다.

<i class="fas fa-comment"></i> 다음 셀은 실행하는 데 시간이 다소 소요될 수 있습니다.

In [ ]:
# Allocate weights and bias for the first layer
w_hd_1 = nd.random_normal(shape=(num_inputs, num_hidden), scale=weight_scale, ctx=ctx)
b_hd_1 = nd.random_normal(shape=num_hidden, scale=weight_scale, ctx=ctx)

두 번째 계층에 대한 파라미터를 할당합니다.

In [ ]:
# Allocate weights and bias for the second layer
w_hd_2 = nd.random_normal(shape=(num_hidden, num_hidden), scale=weight_scale, ctx=ctx)
b_hd_2 = nd.random_normal(shape=num_hidden, scale=weight_scale, ctx=ctx)

출력 계층에 대한 파라미터를 할당합니다.

In [ ]:
# Allocate weights and bias for the output layer
w_output = nd.random_normal(shape=(num_hidden, num_outputs), scale=weight_scale, ctx=ctx)
b_output = nd.random_normal(shape=num_outputs, scale=weight_scale, ctx=ctx)

기울기를 계산할 수 있도록 목록에 파라미터를 추가하려면, 다음번 셀에서 다음 코드를 실행합니다.

In [ ]:
# Add parameters to calculate gradients
params = [w_hd_1, b_hd_1, w_hd_2, b_hd_2, w_output, b_output]


for param in params:
    param.attach_grad()

선형 뉴런으로 다중 계층 네트워크를 정의하면, 해당 네트워크는 선형 함수만 될 수 있습니다. 이는 다음의 이유 때문입니다: $\hat{y} = X \cdot W_1 \cdot W_2 \cdot W_2 = X \cdot W_4 $ for $W_4 = W_1 \cdot W_2 \cdot W3$. 네트워크에서 비선형 속성을 포착하도록 하려면, 계층 끝에 활성화 함수를 추가해야 합니다. 이 경우 ReLU 활성화 함수를 사용할 수 있습니다.

은닉층에 대한 ReLU 활성화 함수를 정의하려면, 다음번 셀에서 다음 코드를 실행합니다.

Sigmoid나 Tanh 등의 다른 활성화 함수를 사용할 수도 있습니다.

In [ ]:
# Define a ReLU activation function for the hidden layer
def relu(X):
    return nd.maximum(X, nd.zeros_like(X))

### Softmax output

네트워크의 출력에서 예측은 숫자 0\~9를 예측하는 배열이 됩니다. 출력 계층을 위한 Softmax 활동 함수는 이미지가 특정 클래스에 속하는 확률을 제공합니다. 예를 들어, 배열의 첫 숫자가 0.65라면, 숫자가 0일 확률이 65%라는 뜻입니다.

Softmax 확률을 새 손실 함수에 전달하는 대신, yhat_linear를 전달하고 softmax_cross_entropy 손실 함수 내에서 softmax와 로그를 모두 한 번에 계산합니다. 이 함수는 LogSumExp 트릭([LogSumExp on Wikipedia](https://en.wikipedia.org/wiki/LogSumExp))와 같은 스마트한 기능을 수행합니다.

In [ ]:
# Use a softmax action function for the output layer
def softmax_cross_entropy(yhat_linear, y):
    return - nd.nansum(y * nd.log_softmax(yhat_linear), axis=0, exclude=True)

## 과제 3: 인공 신경망 모델 정의

이번 과제에서는 인공 신경망, 가중치와 편중치를 학습하기 위한 최적화 도구 및 모델 수행 방식을 평가하는 평가 지표를 정의합니다.

모델을 정의하려면, 새 셀에서 다음을 실행합니다.

In [ ]:
# Define the neural network model
def net(X):

    #  Compute the first hidden layer
    h1_linear = nd.dot(X, w_hd_1) + b_hd_1
    h1 = relu(h1_linear)

    #  Compute the second hidden layer
    h2_linear = nd.dot(h1, w_hd_2) + b_hd_2
    h2 = relu(h2_linear)

    #  Compute the output layer
    yhat_linear = nd.dot(h2, w_output) + b_output
    return yhat_linear

반환된 변수는 여전히 Softmax 함수가 적용되지 않은 선형 변수입니다. 해당 함수는 역 전달 중 발생할 수 있는 수치 안정성 문제를 방지하기 위해 softamx_cross_entropy에 직접 적용됩니다.

가중치와 편중치를 학습하기 위한 최적화 도구를 정의하려면 새 셀에서 다음을 실행합니다.

In [ ]:
# Define the optimizer
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

평가 지표를 정의합니다.

In [ ]:
# Define the evaluation metric
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(ctx).reshape((-1, 784))
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

## 과제 4: 교육 루프 및 모델 평가 실행

이 작업에서는 교육 루프를 실행하고 모델을 평가합니다.

교육 루프 실행을 위한 파라미터를 정의합니다

In [ ]:
# Epochs: Iterations over the full network
epochs = 10

# Learning rate: Speed at which the network learns
learning_rate = 0.001

# Define a smooth constant for the moving loss
smoothing_constant = 0.01

인공 신경망 모델을 학습합니다.

In [ ]:
# Train the neural network model
for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx).reshape((-1, 784))
        label = label.as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label_one_hot)
        loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, moving_loss, train_accuracy, test_accuracy))


**Output:**

Epoch 0. Loss: 0.462392371464, Train_acc 0.8805, Test_acc 0.8831 

Epoch 1. Loss: 0.285959471388, Train_acc 0.919967, Test_acc 0.9194

Epoch 2. Loss: 0.198550129106, Train_acc 0.94725, Test_acc 0.9499

Epoch 3. Loss: 0.159744916748, Train_acc 0.9602, Test_acc 0.958

Epoch 4. Loss: 0.125638222475, Train_acc 0.967033, Test_acc 0.9619

Epoch 5. Loss: 0.101477091803, Train_acc 0.97465, Test_acc 0.9689

Epoch 6. Loss: 0.0901461152782, Train_acc 0.976233, Test_acc 0.9693

Epoch 7. Loss: 0.0763436301528, Train_acc 0.98115, Test_acc 0.9737

Epoch 8. Loss: 0.0693615894988, Train_acc 0.9841, Test_acc 0.9745

Epoch 9. Loss: 0.0573878861228, Train_acc 0.985933, Test_acc 0.9739


### 시각화

테스트 세트에서 무작위 데이터 요소 몇 개를 선택하여 예측과 함께 시각화합니다. 정량적으로는 모델이 더 정확하지만 아래의 이유 때문에 결과를 시각화하는 것이 좋습니다.
* 코드가 실제로 작동하는지 확인 가능
* 모델이 어떤 종류의 실수를 저지르는 경향이 있는지에 대한 직관 제공

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Define the function to do prediction
def model_predict(net,data):
    output = net(data)
    return nd.argmax(output, axis=1)

samples = 10

mnist_test = mx.gluon.data.vision.MNIST(train=False, transform=transform)

# let's sample 10 random data points from the test set
sample_data = mx.gluon.data.DataLoader(mnist_test, samples, shuffle=True)
for i, (data, label) in enumerate(sample_data):
    data = data.as_in_context(ctx)
    im = nd.transpose(data,(1,0,2,3))
    im = nd.reshape(im,(28,10*28,1))
    imtiles = nd.tile(im, (1,1,3))
    
    plt.imshow(imtiles.asnumpy())
    plt.show()
    pred=model_predict(net,data.reshape((-1,784)))
    print('model predictions are:', pred)
    print('true labels :', label)
    break

보이지 않는 데이터에 대한 모델의 작동 방식을 확인하려면 새 셀에서 다음을 실행하여 Jupyter 노트북에 그릴 수 있는 캔버스가 포함된 html 페이지를 다운로드합니다.

Jupyter 노트북에는 매직 명령어\(Magic Command\)가 내장되어 있어 Jupyter 노트북 셀에서 Bash 명령어 및 기타 명령어를 실행할 수 있습니다. 자세한 내용은 [Magic Commands](http://ipython.readthedocs.io/en/stable/interactive/magics.html)를 참조하십시오.

In [ ]:
%%bash
wget http://us-west-2-tcdev.s3.amazonaws.com/<% KAIZEN_S3_LAB_PREFIX %>/scripts/mnist.html

다음 셀을 실행하여 모델을 평가하기 위한 HTML 캔버스를 생성합니다.

마우스를 사용하여 표시된 사각형 안에 숫자를 그린 다음 **Classify**를 클릭합니다.

마우스로 사각형 안에 숫자를 적고 Classify 버튼을 눌러 분류 결과를 확인해 봅시다.

In [ ]:
# Create an HTML canvas to evaluate the model
from IPython.display import HTML
import cv2
import numpy as np
import base64

def classify(img):
    img = base64.b64decode(img[len('data:image/png;base64,'):])
    img = cv2.imdecode(np.fromstring(img, np.uint8),-1)
    img = cv2.resize(img[:,:,3], (28,28))
    img = nd.array(img).as_in_context(ctx).reshape((-1, 784)).astype(np.float32)/255
    return int(nd.argmax(net(img), axis=1).asnumpy()[0])

HTML(filename = "mnist.html")

## 도전 과제

- 네트워크 크기를 5개의 계층으로 변경해 보십시오.
- 숨겨진 단위 수를 256으로 변경해 보십시오.

## 실습 완료

축하합니다! 본 실습을 완료했습니다. 실습 환경을 정리하려면 다음을 수행하십시오.

- 노트북 파일을 닫습니다.
- **Quit** 을 클릭하여 Jupyter 노트북에서 로그아웃한 다음 탭을 닫습니다.
- AWS Management Console에서 로그 아웃하려면 콘솔 맨 위에 있는 **awsstudent**을 클릭한 후 **Sign Out**을 클릭합니다.
- **End Lab** 을 클릭하여 Qwiklabs에서 실습 세션을 종료합니다.